# **2) Mean Data**

##### *À quoi ressemble notre df nettoyé ?*

##### <span style='color:red'>**Modifier data ici ↓**</span>

In [1]:
import pandas as pd
import re

cleaned_data_louer = pd.read_csv('1_useful_data/1_cleaned_data/Cleaned_Immobilier_ch_louer.csv')
cleaned_data_louer.head(7)

,ville,canton,prix,metres_carres,type,nbr_pieces
0,Selzach,SO,990,72,Appartement,3.5
1,Vernier,GE,2600,30,Appartement meublé,NaN
2,Paradiso,TI,3940,30,Appartement meublé,1.0
3,Zürich,ZH,3185,30,Appartement meublé,1.0
4,Vernier,GE,2650,48,Appartement,3.0
5,Romanel-sur-Lausanne,VD,1960,82,Appartement,3.5
6,Neuchâtel,NE,2490,150,Villa,6.0


---

##### **Définition de la fonction Prix_Metre_Carre**

In [2]:
def prix_metre_carre(data):
    #Divise le prix par le nombre de mètres carrés pour obtenir le prix au mètre carré pour chaque ligne d'un bien immobilier
    data['prix_metre_carre'] = data['prix'] / data['metres_carres']

    #Compte le nombre de biens immobiliers pour chaque ville
    data['canton'] = data['canton'].fillna('')
    property_count = data.groupby(['ville', 'canton']).size()
    
    #Groupe par ville et calcule le prix moyen par mètre carré pour chaque ville
    mean_price_per_sq_meter = data.groupby(['ville', 'canton']).apply(lambda x: x['prix'].sum() / x['metres_carres'].sum())

    #On ne prends pas en compte les villes avec 1 ou 0 bien immobilier
    mean_price_per_sq_meter[property_count < 2] = 0


    #Si une ville a entre 2 et 3 biens immobiliers et que la moyenne de la variation en % du prix au mètre carré entre ces biens est supérieure à 8 %, on l'ignore
    two_to_four_properties = property_count[(property_count >= 2) & (property_count <= 3)].index
    for (ville, canton) in two_to_four_properties:

        # Extrait les prix au mètre carré pour les biens de la ville et canton
        prix_metre_carre_ville = data[(data['ville'] == ville) & (data['canton'] == canton)]['prix_metre_carre']

        diff = []
        #Calcul de toutes les variations entre le prix au mètre carré des biens d'une même ville
        #Itère sur les indices des éléments de la série 'prix_metre_carre_ville'
        for i in range(len(prix_metre_carre_ville)):
            #À l'intérieur de la première boucle, on a une autre boucle for imbriquée qui commence à l'index suivant i+1 jusqu'à la fin de la série prix_metre_carre_ville. Cela permet de parcourir toutes les paires possibles de valeurs de la série.
            for j in range(i+1, len(prix_metre_carre_ville)):
                diff.append(abs((prix_metre_carre_ville.iloc[i] - prix_metre_carre_ville.iloc[j]) / prix_metre_carre_ville.iloc[i]) * 100)

        #Calcul de la moyenne des variations en %
        if len(diff) > 0: #pour éviter une erreur si comptage=0
            mean_variation = sum(diff) / len(diff)
        else:
            mean_variation = 0
        'print(f"Ville: {ville}, Moyenne de la variation des prix au mètre carré : {mean_variation:.2f}%")'

        #Vérifie si la moyenne des variations dépasse 8%
        if mean_variation > 8:
            mean_price_per_sq_meter.loc[ville] = 0


    #Crée un DataFrame avec la moyenne et réinitialise l'index pour avoir 'ville' en colonne
    result = mean_price_per_sq_meter.reset_index()
    
    # Renommer les colonnes
    result.columns = ['ville', 'canton' ,'prix_metre_carre_moyen']


    return result


##### **Définition de la fonction Nbr_villes_utilisables**

In [3]:
def nbr_villes_utilisables(dataframe):
    
    non_zero_avg = dataframe[dataframe['prix_metre_carre_moyen'] != 0]
    count_rows = len(non_zero_avg)

    return count_rows

---

### ***Mean Data Immobilier.ch Louer*** 

In [4]:
prix_metre_carre_mensuel = prix_metre_carre(cleaned_data_louer)
prix_metre_carre_mensuel.head(20)

,ville,canton,prix_metre_carre_moyen
0,Aadorf,TG,15.049751
1,Aarau,AG,0.000000
2,Aarburg,AG,16.863430
3,Aefligen,BE,0.000000
4,Aire-la-Ville,GE,0.000000
5,Allaman,VD,0.000000
6,Alle,JU,13.584906
7,Allschwil,BL,20.031350
8,Alpnach,OW,16.768559
9,Altnau,TG,0.000000


In [5]:
#Nombre de villes avec une moyenne à la location utilisable (Louer)

nb_v_utiles_louer = nbr_villes_utilisables(prix_metre_carre_mensuel)
print(nb_v_utiles_louer)

374


In [6]:
#Création du nouveau fichier CSV avec la moyenne par ville du mètre carré à la location
prix_metre_carre_mensuel.to_csv('1_useful_data/2_mean_data/Prix_metre_carre_louer.csv', index=False)

---

### ***Mean Data Immobilier.ch Acheter*** 
##### <span style='color:red'>**Modifier data ici ↓**</span>

In [7]:
cleaned_data_acheter = pd.read_csv('1_useful_data/1_cleaned_data/Cleaned_Immobilier_ch_acheter.csv')
cleaned_data_acheter.head()

,ville,canton,prix,metres_carres,type,nbr_pieces
0,Ollon,VD,2650000,180,Chalet,5.0
1,Leytron,VS,555000,87,Appartement,3.5
2,Val de Bagnes,VS,3500000,146,Appartement,5.5
3,Mont-Noble,VS,755000,146,Chalet,5.5
4,Liestal,BL,1840000,150,Appartement,4.5


In [8]:
prix_metre_carre_achat = prix_metre_carre(cleaned_data_acheter)
prix_metre_carre_achat.head(20)

,ville,canton,prix_metre_carre_moyen
0,Aarburg,AG,6699.134199
1,Aclens,VD,8559.539052
2,Acquarossa,TI,4433.638444
3,Adligenswil,LU,0.000000
4,Aesch,ZH,0.000000
5,Aeschi,SO,0.000000
6,Aeschi bei Spiez,BE,0.000000
7,Agno,TI,0.000000
8,Aire-la-Ville,GE,9385.931916
9,Airolo,TI,1830.357143


In [9]:
#Nombre de villes avec une moyenne à l'achat utilisable (Acheter)

nb_v_utiles_acheter = nbr_villes_utilisables(prix_metre_carre_achat)
print(nb_v_utiles_acheter)

586


In [10]:
#Création du nouveau fichier CSV avec la moyenne par ville du mètre carré à l'achat
prix_metre_carre_achat.to_csv('1_useful_data/2_mean_data/Prix_metre_carre_acheter.csv', index=False)

---

##### *On va voir combien de villes ont une moyenne à l'achat et à la location*

In [11]:
#DF sans les villes non utilisables (<3 annonces)
villes_valides_acheter = prix_metre_carre_achat[prix_metre_carre_achat['prix_metre_carre_moyen'] != 0.0]
villes_valides_louer = prix_metre_carre_mensuel[prix_metre_carre_mensuel['prix_metre_carre_moyen'] != 0.0]


# Utilisation de merge pour obtenir les villes communes
merged_df = pd.merge(villes_valides_acheter, villes_valides_louer, on=['ville', 'canton'],  how='inner')

# Compter le nombre de villes communes
print(len(merged_df))

244


ㅤ

---